<span type="title">Deploy and Security</span> | <span type="update">2018-09-26</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Servlet 和 JSP 应用程序的部署和安全性问题。</p></span>

# Web 程序部署

## 文件结构

TOMCAT 是部署 Web 程序的容器。其结构如下：

- / 此目录下保存着所有可以公开访问的 HTML 和 JSP 页面
- /META-INF 此目录下保存着连接到外部的类和库的声明，不可公开访问
- /WEB-INF 此目录下保存着tld说明符（此目录），Java字节码（.class子目录），tag标记（tag子目录），库（lib目录下的jar文件中）。所有文件不可公开访问

所有在 META-INF，WEB-INF 之内的文件夹以及递归文件夹所有文件均不可通过URL直接访问。此外，tag标记还可以放在 /WEB-INF/tag/ 目录下的子目录中，以及jar库的META-INF目录及其子目录中，tld说明符也可以放在 /WEB-INF 的目录以及其任意子目录下，以及jar库的META-INF目录及其子目录中。

当容器需要一个类，其搜索顺序为 /WEB-INF/class目录下 --> /WEB-INF/lib/jar 包中 ---> TOMCAT_HOME/lib 中。

## URL 匹配

URL 映射在 DD 中进行配置。对于 URL 而言，其代表了虚拟的映射，一般分为： `http://host/folder` 和 `http://host/folder/file` 这两种请求情况，其中 file 类型可以加后缀，也可以不加。这就造成了混淆，因为从外观上看它们没有什么区别。Servlet 的映射规则如下：

对于任意的请求，都依次进行以下三种的匹配，如果在任意匹配中找到，就不往下进行。这三种匹配是 完全限定、目录匹配以及精确匹配。

完全限定：`/xxx/xxxx.do` 或者 `/xxx/xxxx` 需要有第一个斜线，可选文件名后缀

目录匹配：`/xxx/*` 或者 `/xxx/` 需要有第一个斜线，必须写星号以及其前面的那个斜线，因为不写的话，和完全限定没有区别，不知道是不是目录。也可以使用首尾两个斜线表示目录。

文件匹配：`*.do` 必须写后缀以及星号。不能写 `/*.do` 这样，容器不会进行匹配。

对于 URL 是目录而言， `http://host/folder/` 直接从目录匹配开始进行。对于 URL 看不出是目录而言，`http://host/file_or_folder` 从完全限定的文件开始匹配，找不到再找目录。

此外，需要注意，对于目录匹配，如果 URL `/a/b/c/d/` 同时满足 `/a/b/c/*` 和 `/a/b/*`, 那么取最特殊的那一个。

举例：现有 URL `a/b` 和 `a/b/` ，有 Servlet 规则 `a/b` 和 `a/b/*`，那么，`a/b` 的 URL 匹配 `a/b`，这是因为从完全限定开始匹配，正好找到，就不继续找目录了。对于 URL `a/b/` 这是一个目录，直接找 `a/b/*` 这个目录，得到匹配。

## 默认文件和错误文件

如果用户对于一个目录进行访问，那么哪个文件响应请求呢？对于物理映射的JSP和目录而言，有这样的问题。我们可以设置欢迎界面，当对物理目录进行访问，会自动从DD中进行选择，如果第一项不存在，则寻找并返回第二项，依次类推。

```xml
<welcome-file-list>
    <welcome-file>index.html</welcome-file>
    <welcome-file>default.jsp</welcome-file>
</welcome-file-list>
```

对于错误文件，在上一章有所介绍，可以根据相应代码，也可以根据Java错误类型返回。

```xml
<error-page>
    <error-code>404</error-code>
    <location>/404.jsp</location>
</error-page>
<error-page>
    <exception-type>java.lang.Throwable</exception-type>
    <location>/exception.jsp</location>
</error-page>
```

注意，在 DD 中所有的 location 以及对物理资源的指定都需要使用 / 绝对目录，这代表着 APP 的目录。

## Servlet 启动顺序

Servlet 启动顺序可以由 `load-on-startup` 标签指定：

```xml
<servlet>
    <servlet-name>LearnEL</servlet-name>
    <jsp-file>/learnEL.jsp</jsp-file>
    <load-on-startup>1</load-on-startup>
</servlet>
```

其中数字没有含义，但是其大小有。小于等于0无意义，等于没有写。数字越大，启动越靠后。

## WAR 文件部署

war 文件是打包好的 WEB 程序，采用 jar 标准打包，只是改了个名字叫做 war，其包含所有的类、库、资源文件以及jsp、tld、tag、html、css、xml文件。对 jar/war 中的资源文件访问需要使用 JavaSE 标准的 getResourceAsStream() 来访问。war 的名称会被用作 app 的 url 路径以及名称，可以在容器中更改这一设置。

# Servlet 和 JSP 安全

## 